In [1]:
import os, pandas as pd, requests

DATA_RAW = "../data/raw"
DATA_INTERIM = "../data/interim"
os.makedirs(DATA_RAW, exist_ok=True)
os.makedirs(DATA_INTERIM, exist_ok=True)

In [22]:
from io import StringIO

# Paths
DATA_RAW = "../data/raw"
os.makedirs(DATA_RAW, exist_ok=True)

# API endpoint
base_url = "https://ffiec.cfpb.gov/v2/data-browser-api/view/csv"

params = {
    "years": "2022",
    "states": "CA",
    "loanTypes": "1",      # conventional
    "propertyTypes": "1",  # 1–4 family, site-built
    "occupancyTypes": "1", # primary
    "lienStatuses": "1",    # first lien
    "loan_purposes": "1"    # purchase loans
}

# Add User-Agent header to avoid 403
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

resp = requests.get(base_url, params=params, headers=headers)
resp.raise_for_status()

# Read CSV from response 
df_raw = pd.read_csv(StringIO(resp.text))

# Save raw pull
raw_out = os.path.join(DATA_RAW, "hmda_ca2022_raw.csv")
df_raw.to_csv(raw_out, index=False)

print("Rows, Cols:", df_raw.shape)
df_raw.head(3)

/var/folders/ty/9hbmbw6113l8wpm40mbk4qk40000gn/T/ipykernel_66159/2934110892.py:30: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(StringIO(resp.text))


Rows, Cols: (611698, 99)


,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2022,549300FGXN1K3HLB1R50,40900,CA,6061.0,6.061024e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4361,11.10,102400,97.88,2252,2634,0
1,2022,549300FGXN1K3HLB1R50,40900,CA,6067.0,6.067009e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,8897,54.51,102400,173.83,2287,2568,21
2,2022,549300FGXN1K3HLB1R50,31084,CA,6037.0,6.037481e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4183,57.54,91100,186.97,696,1045,65


In [27]:
DATA_INTERIM = "../data/interim"
os.makedirs(DATA_INTERIM, exist_ok=True)

# From previous cell
cohort = df_raw.copy()

# Convert race and ethnicity indicators to numbers
int_fields = [
    "applicant_race-1", "co-applicant_race-1",
    "applicant_ethnicity-1", "co-applicant_ethnicity-1",
    "applicant_race_observed"
]
for col in int_fields:
    cohort[col] = pd.to_numeric(cohort[col], errors="coerce").astype("Int64")

# Define race indicator

def classify_race(row):
    app_race = row["applicant_race-1"]
    co_app_race = row["co-applicant_race-1"]
    app_eth = row["applicant_ethnicity-1"]
    co_app_eth = row["co-applicant_ethnicity-1"]
    app_obs = row["applicant_race_observed"]

    # In cases where the co-applicant is identified as ‘White’ and the applicant is of a different race, we reclassify the applicant’s race as ‘White’ 
    if (pd.notna(app_race) and app_race == 5) or (pd.notna(co_app_race) and co_app_race == 5):
        return "White"

    # Hispanic as a distinct group regardless of race
    if (pd.notna(app_eth) and app_eth in [1, 11, 12, 13, 14]) or \
       (pd.notna(co_app_eth) and co_app_eth in [1, 11, 12, 13, 14]):
        return "Hispanic or Latino"

    # Classify race group based on the primary applicant's race
    if pd.notna(app_race):
        if app_race == 1:
            return "American Indian or Alaska Native"
        if app_race in [2] + list(range(21, 28)):
            return "Asian"
        if app_race == 3:
            return "Black or African American"
        if app_race in [4, 41, 42, 43, 44]:
            return "Pacific Islander"

    # Classify race group based on the primary applicant's race
    if pd.notna(app_obs):
        if app_obs == 1:
            return "American Indian or Alaska Native"
        if app_obs in [2] + list(range(21, 28)):
            return "Asian"
        if app_obs == 3:
            return "Black or African American"
        if app_obs in [4, 41, 42, 43, 44]:
            return "Pacific Islander"

    return "Other / Unknown"

# Apply race classification
cohort["race_group"] = cohort.apply(classify_race, axis=1)

# Drop loans with missing/undisclosed race
cohort = cohort[cohort["race_group"] != "Other / Unknown"].copy()

# Add relative income to area median to control for local affordability
cohort["income_ratio"] = cohort["income"] / cohort["ffiec_msa_md_median_family_income"]

# Bucket income into three classification CRA & Apgar (2007)
def income_bucket(ratio):
    if pd.isna(ratio):
        return "Unknown"
    elif ratio < 0.8:
        return "Low (<80%)"
    elif ratio <= 1.2:
        return "Moderate (80–120%)"
    else:
        return "High (>120%)"

cohort["income_bucket"] = cohort["income_ratio"].apply(income_bucket)

# Data cleaning
# 1. Exclude intro-rate loans
cohort = cohort[cohort["intro_rate_period"].isna()]

# 2. Exclude extreme LTV values (potential data entry error)
cohort = cohort[pd.to_numeric(cohort["loan_to_value_ratio"], errors="coerce") < 200]

# 3. Drop NAs for key predictors
cohort = cohort[
    cohort["rate_spread"].notna()
    & cohort["loan_to_value_ratio"].notna()
    & cohort["income"].notna()
    & (cohort["income"] > 0)
    & cohort["debt_to_income_ratio"].notna()
    & (cohort["debt_to_income_ratio"] != "")
]

# 4. Clean DTI
def clean_dti(val):
    if pd.isna(val):
        return None
    val = str(val).strip()
    mapping = {
        "<20%": 15,
        "20%-<30%": 25,
        "30%-<36%": 33,
        "50%-60%": 55,
        ">60%": 65,
        "NA": None,
        "Exempt": None
    }
    if val in mapping:
        return mapping[val]
    try:
        return float(val.strip("%"))
    except:
        return None

cohort["dti_clean"] = cohort["debt_to_income_ratio"].apply(clean_dti)

# Reset index
cohort.reset_index(drop=True, inplace=True)

# Save interim dataset 
out_path = os.path.join(DATA_INTERIM, "hmda_ca2022_conventional.csv")
cohort.to_csv(out_path, index=False)

# Print summary 
print(f"Cohort saved: {out_path}")
print("Rows:", len(cohort))
print("\nRace group counts:")
print(cohort["race_group"].value_counts())

Cohort saved: ../data/interim/hmda_ca2022_conventional.csv
Rows: 260797

Race group counts:
race_group
White                               149860
Asian                                83209
Hispanic or Latino                   13993
Black or African American            12268
American Indian or Alaska Native       787
Pacific Islander                       680
Name: count, dtype: int64
